## Clear and Reformat Population Dataset

In [302]:
import openpyxl
import pandas as pd
import numpy as np

In [303]:
workbook = openpyxl.load_workbook('co-est2023-pop.xlsx')
worksheet = workbook.active

In [304]:
start_row = 6
end_row = 3149
columns = ['A', 'B', 'C', 'D', 'E', 'F']

data = {col: [] for col in columns}

for row in range(start_row, end_row + 1):
    for col in columns:
        cell_value = worksheet[col + str(row)].value
        data[col].append(cell_value)

df = pd.DataFrame(data)

In [305]:
new_column_names = ['County', 'Estimates Base', '2020', '2021', '2022', '2023']
df.columns = new_column_names
df

,County,Estimates Base,2020,2021,2022,2023
0,".Autauga County, Alabama",58809,58915,59203,59726,60342
1,".Baldwin County, Alabama",231768,233227,239439,246531,253507
2,".Barbour County, Alabama",25229,24969,24533,24700,24585
3,".Bibb County, Alabama",22301,22188,22359,21986,21868
4,".Blount County, Alabama",59130,59107,59079,59516,59816
...,...,...,...,...,...,...
3139,".Sweetwater County, Wyoming",42271,42197,41626,41374,41249
3140,".Teton County, Wyoming",23323,23379,23605,23297,23232
3141,".Uinta County, Wyoming",20445,20457,20681,20727,20745
3142,".Washakie County, Wyoming",7679,7657,7719,7724,7710


In [306]:
### reformat and clear dataset
county_names =  df['County'].apply(lambda x: (x.split(',')[0][1:]))
state_names = df['County'].apply(lambda x: (x.split(',')[1]))
county_state_names = df['County'].apply(lambda x: (x[1:]))
df['County'] = county_names
df['State'] = state_names
df['County_State'] = county_state_names

In [307]:
### add fips number to the existing dataset
county_df = pd.read_csv('county_fips_master.csv', encoding='ISO-8859-1')
state_df = pd.read_csv('state_fips_master.csv', encoding='ISO-8859-1')

In [308]:
### check non-machtcing cases
# Create a boolean mask where False indicates the county is in county_df
mask = ~df['County'].isin(county_df['county_name'])

# Sum the non-matching entries (optional, if you want the count)
non_matching_count = np.sum(mask)

# Get the indices of non-matching entries
non_matching_indices = df.index[mask]

# If you need to see which are these counties
non_matching_counties = df['County'][mask]
non_matching_counties


72                                 Chugach Census Area
73                            Copper River Census Area
309                            Capitol Planning Region
310                 Greater Bridgeport Planning Region
311     Lower Connecticut River Valley Planning Region
312                   Naugatuck Valley Planning Region
313           Northeastern Connecticut Planning Region
314                    Northwest Hills Planning Region
315          South Central Connecticut Planning Region
316           Southeastern Connecticut Planning Region
317                Western Connecticut Planning Region
1144                                    LaSalle Parish
1804                                   Doña Ana County
Name: County, dtype: object

In [309]:
mask = county_df['county_name'].str.contains('otero', case=False)

# Apply the mask to get the DataFrame rows where county names contain "census" (case-insensitive)
counties_containing_census = county_df[mask]
counties_containing_census

,fips,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name
290,8089,Otero County,CO,Colorado,Otero County CO,50.0,4.0,8.0,8.0,89.0,4-8-8-89,West,Mountain
1816,35035,Otero County,NM,New Mexico,Otero County NM,50.0,4.0,8.0,35.0,35.0,4-8-35-35,West,Mountain


In [310]:
### discard non matching cases
df_clear = df[~mask]
df_clear.shape
county_df['county_state_name'] = county_df['county_name'] + ', ' +  county_df['state_name']
df_combine = pd.merge(df_clear, county_df[['county_state_name', 'fips']], left_on='County_State', right_on='county_state_name', how='inner')
df_combine.drop(columns='county_state_name', inplace=True)

/tmp/ipykernel_32000/3089728510.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_clear = df[~mask]


In [311]:
### process fips into five-digit strings
df_combine['state_county_fips'] = df_combine['fips'].astype(int).astype(str).apply(lambda x: '0' + x if len(x) < 5 else x)
df_combine.drop(columns='fips', inplace=True)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
### drop estimates base
df_combine.drop(columns='Estimates Base', inplace=True)

In [ ]:
df_combine.columns = ['county', '2020', '2021', '2022', '2023', 'state', 'county_state', 'id']

In [ ]:
### log population
df_combine['2020'] = np.log(df_combine['2020'])
df_combine['2021'] = np.log(df_combine['2021'])
df_combine['2022'] = np.log(df_combine['2022'])
df_combine['2023'] = np.log(df_combine['2023'])

In [ ]:
df_combine.to_csv('state_county_population.csv', index=False)